In [1]:
from SimCsstLens.LensPop.Population import LensPopulation
from SimCsstLens import CosmologyDistance as CD
import math
import h5py
import numpy as np 
from multiprocessing import Pool
import os
from astropy.table import Table, vstack

this_cosmos = CD.CosmosDist(Om0=0.3, Ode0=0.7, h=0.7)
lens_pop = LensPopulation(
    vdisp_floor=100, 
    zl_max=2.0, 
    cosmo_dist=this_cosmos,
    src_catalog_type='lsst',
    bands=['g', 'r', 'i', 'z'],
)
sky_frac = 9.0/41252.96
N_etgs = lens_pop.dfl_pop.number_of_etgs(sky_frac=sky_frac) #ideal lenses
Nsamples_per_draw = 5000
Ndraw = math.ceil(N_etgs/Nsamples_per_draw)

In [2]:
current_dir = os.getcwd()
def return_detected(count_draw, stack=False):
    table = Table()

    fn = h5py.File(f"./samples/lenses_{count_draw}.hdf5", "r")
    src_thetaE = fn['source/thetaE'][()]
    for ii in range(src_thetaE.shape[0]):
        table[f'thetaE_s{ii}'] =  src_thetaE[ii, :]
        table[f'mass_s{ii}'] = fn['source/einstein_mass'][()][ii, :]
        table[f'z_s{ii}'] = fn['source']['z'][()][ii, :]
        table[f're_s{ii}'] = fn['source']['Re'][()][ii, :]
        table[f'q_s{ii}'] = fn['source']['q'][()][ii, :]
        table[f'pa_s{ii}'] = fn['source']['pa'][()][ii, :]
        table[f'x_s{ii}'] = fn['source']['xs'][()][ii, :]
        table[f'y_s{ii}'] = fn['source']['ys'][()][ii, :]
        table[f'mag_g_s{ii}'] = fn['source']['app_mag_g'][()][ii, :]
        table[f'mag_r_s{ii}'] = fn['source']['app_mag_r'][()][ii, :]
        table[f'mag_i_s{ii}'] = fn['source']['app_mag_i'][()][ii, :]
        table[f'mag_z_s{ii}'] = fn['source']['app_mag_z'][()][ii, :]

    table['vdisp_l'] = fn['deflector/vdisp'][()]
    table['re_l'] = fn['deflector/Re'][()]
    table['q_l']= fn['deflector/q'][()]
    table['z_l'] = fn['deflector/z'][()]
    table['mag_g_l'] = fn['deflector/app_mag_g'][()]
    table['mag_r_l'] = fn['deflector/app_mag_r'][()]
    table['mag_i_l'] = fn['deflector/app_mag_i'][()]
    table['mag_z_l'] = fn['deflector/app_mag_z'][()]

    SNR = fn['Obs/SNR'][()] #shape: [nsrc, nband, n_ideal_lens]
    table['SNR_g'] = SNR[0,0,:]
    table['SNR_r'] = SNR[0,1,:]
    table['SNR_i'] = SNR[0,2,:]
    table['SNR_z'] = SNR[0,3,:]
    table['SNR_stack'] = SNR[0,-1,:]
    SNR_griz = SNR[0,0:-1,:]
    indices = np.argmax(SNR_griz, axis=0)
    table['best_band'] = np.array(['g', 'r', 'i', 'z'])[indices] 
    
    mu = fn['Obs/magnification'][()] #shape: [nsrc, nband, n_ideal_lens]
    table['mu'] = mu[0,-1,:]
    
    ring_cond = fn['Obs/ring_cond'][()] #shape: [nsrc, nband, n_ideal_lenses]
    table['if_ring'] = ring_cond[0,-1,:]

    obs_cond = fn['Obs/detect_cond'][()] #shape: [nsrc, nband, n_ideal_lenses, 3]
    
    fn.close()


    #stack band
    this_obs_cond = obs_cond.astype('int')
    this_obs_cond = this_obs_cond[0, -1, :, :] #src-0, band-stack, nlens, 3-condition
    this_obs_cond = np.sum(this_obs_cond, axis=1) #shape: [nlens]
    bool_cond = (this_obs_cond==3)
    table['if_obs_stack'] = bool_cond
        
    #individual band
    this_obs_cond = obs_cond.astype('int')
    this_obs_cond = this_obs_cond[:, :-1, :, :] #remove stack band
    cond = np.sum(this_obs_cond, axis=3)[0,:,:] #cond: [nband, nlens]
    bool_cond = (cond==3).astype('int') #cond: [nband, nlens]
    bool_cond = (np.sum(bool_cond, axis=0)>0) #bool_cond" [nlens]
    table['if_obs_single'] = bool_cond
    
    n_detect_lens = len(np.where(bool_cond)[0])
    n_ideal_lens = src_thetaE.shape[1]
    
    table['noise_seed'] = np.arange(n_ideal_lens, dtype='int')
    
    return n_detect_lens, n_ideal_lens, table

n_detect_lens, n_ideal_lens, table = return_detected(0, stack=True)
table[0:5]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,best_band,mu,if_ring,if_obs_stack,if_obs_single,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,bool,bool,bool,int64
0.34654967935920145,28890342263.21468,0.8095,0.09701996081216313,0.6522624003814266,77.19658016364643,0.10697759955269898,-0.1348101644429451,25.8061,25.5448,24.8475,24.6583,152.5715256671548,0.5758979754401826,0.42632008956604706,0.3459937760977172,22.58589938414749,20.63553064011652,19.97824896979034,19.6037103868216,87.96799126643502,72.83147846295138,101.60521833466625,77.7448962702923,169.17728939537756,i,3.937739693064625,True,True,True,0
0.2557068522821534,29079647537.00696,2.1366,0.037780093373152655,0.35251909574512363,74.17277626001191,0.18696460723977654,0.08581141893484165,27.523,27.6056,27.6722,27.6067,141.81106549973066,0.39878390678353187,0.4996854428055003,0.8806004717223257,28.042244719218807,25.266241907914956,24.015018470475493,22.792025961246477,21.264671740443973,15.316449226653688,11.314647273221146,7.375835447137116,27.515506677382643,g,2.1844944997751257,False,False,False,1
0.3738327001582531,53231442397.31827,2.1788,1.2135559184982019,0.4680509090174003,37.061098401753476,0.32858310118097916,-0.1037231541219037,26.4399,26.5609,26.5627,26.1826,161.75715743821627,0.28313638045910544,0.9073631358777517,0.7822600972480143,26.692565426202428,24.297405589305292,22.890322311540718,21.98199879323414,4.664597731039863,3.22901234623794,2.6483985635694385,2.5327296160649286,6.50509006809372,g,1.4999421242437343,False,False,False,2
0.7455501759359489,156332898231.25385,2.4373,0.07426816490162264,0.8090153964687051,179.9316575561918,0.6107874969789586,-0.13605541844986335,26.9317,26.9879,27.2505,27.1876,206.07659771385698,0.3023541177212386,0.8096571334510696,0.6146119873597595,23.90762956548182,22.013107353049342,20.647522806020806,20.067130586616518,32.58320049165174,24.368222044203044,15.566117832932,10.647992533911157,41.83170056833742,g,2.221986909503503,False,False,False,3
0.46695473972695534,49188008819.938156,4.3409,0.3085054776019881,0.3619119600519518,91.80629425401443,0.14676374401541392,0.37055624087124306,28.3052,27.3211,26.9087,26.874,149.78875311676168,0.41188159237936367,0.7391351835442175,0.5527347056285474,24.680431727514705,22.839693754819542,21.60236063720239,21.04994608698747,3.858396872724322,7.23650190599813,8.393873809720139,5.74027926633498,12.776160809437483,i,2.9268067129270254,False,False,False,4


In [3]:
pool = Pool(processes=64)
vars = list(zip(list(range(Ndraw)), [True]*Ndraw))
results = pool.starmap(return_detected, vars)

n_detect_lens = [item[0] for item in results]
n_ideal_lens = [item[1] for item in results]
n_detect_lens = sum(n_detect_lens)
n_ideal_lens = sum(n_ideal_lens)

tables = [item[2] for item in results]
stacked_table = vstack(tables)
stacked_table.write('lens_samples.fits', overwrite=True)

In [4]:
len(stacked_table)

2675

In [5]:
mask = stacked_table['if_obs_stack'].data
table_detect = stacked_table[mask]
table_detect.write("catalog_csv/csst_udf_stack.csv")
len(table_detect)

603

In [6]:
mask = stacked_table['if_obs_single'].data
table_detect = stacked_table[mask]
table_detect.write("catalog_csv/csst_udf_single.csv")
len(table_detect)

520